In [6]:
api_key = 'Q2ScRTFxPN3auqFzSwTARZjwyReBBqvGj6o62mwHs8dE3sq6ghkrwbcmAHIIFPB9'
secret = 'YbYojfUG1Ysk1Wg6yAHt1fZmQ47Jq8fKBkIXu3JZn41XzK1G9LnJHRiIyYhx8zWH'

In [ ]:
from binance.client import Client

from binance.spot import Spot

from datetime import datetime, timedelta
import pandas as pd

# Initialize the Binance Spot client
client = Spot()


def get_from_api_historical_prices(list_of_symbol):
    historical_prices  = pd.DataFrame()

    for symbol in list_of_symbol:
        # Define the symbol and interval
        interval = Client.KLINE_INTERVAL_1MINUTE

        # Calculate the start and end times (6 months ago from now)
        end_time = client.get_server_time()['serverTime']
        start_time = end_time - (1 * 30 * 24 * 60 * 60 * 1000)  # 6 mois en millisecondes

        # Fetch historical klines data
        klines = client.get_historical_klines(symbol, interval, start_time, end_time)

        # Define the columns for the DataFrame
        spot_kline_cols = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']

        # Convert the klines data to a DataFrame
        df = pd.DataFrame(klines, columns=spot_kline_cols)
        df['Symbol'] = symbol
        df['timestamp'] = pd.to_datetime(df['open_time'], unit='ms')
        historical_prices = pd.concat([historical_prices, df])

    return historical_prices

In [7]:
from binance.client import Client
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
client = Client(api_key= api_key, api_secret=secret)

def get_from_api_historical_prices(list_of_symbol):
    historical_prices = pd.DataFrame()

    for symbol in list_of_symbol:
        interval = '1d'  # Intervalle de 24 heures pour l'API

        # Obtenir le temps de fin (temps actuel)
        end_time = datetime.now()
        # Calculer le temps de début (3 ans)
        start_time = end_time - timedelta(days=36*31)  # Environ 3 ans

        # Convertir start_time et end_time en millisecondes pour l'API
        start_time_str = start_time.strftime('%Y-%m-%d %H:%M:%S')
        end_time_str = end_time.strftime('%Y-%m-%d %H:%M:%S')

        # Fetch historical klines data
        klines = client.get_historical_klines(symbol, interval, start_time_str, end_time_str)

        # Define the columns for the DataFrame
        spot_kline_cols = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']

        # Convert the klines data to a DataFrame
        df = pd.DataFrame(klines, columns=spot_kline_cols)
        df['Symbol'] = symbol
        df['timestamp'] = pd.to_datetime(df['open_time'], unit='ms')
        df.set_index('timestamp', inplace=True)  # Optionnel: Utiliser 'timestamp' comme index
        historical_prices = pd.concat([historical_prices, df])
        
        historical_prices['close'] = historical_prices['close'].astype(float)
        historical_prices['timestamp'] = pd.to_datetime(historical_prices['open_time'], unit='ms')

    return historical_prices

# Exemple d'utilisations
symbols = ['BTCEUR','ETHEUR','DOGEEUR'] 
#symbols = ['BTCEUR'] # Liste des symboles que vous souhaitez interroger
all_symbols = get_from_api_historical_prices(symbols)

In [8]:
all_symbols.to_csv('../out/symbols_prices.csv', header =True, sep=';', encoding='utf-8', index=False)

In [4]:
all_symbols.Symbol.value_counts()

Symbol
BTCEUR     744
ETHEUR     744
DOGEEUR    744
Name: count, dtype: int64